In [1]:
import osmnx as ox
import pandas as pd
import numpy as np

In [2]:
# longitude is x, latitude is y

G = ox.graph_from_place('San Francisco, CA, USA', network_type='drive')
nodes = ox.graph_to_gdfs(G, edges=False)

In [3]:
temp_cols = list(nodes.columns)
temp_cols[temp_cols.index('x')] = 'lng'; temp_cols[temp_cols.index('y')] = 'lat'
nodes.columns = temp_cols
nodes = nodes.sort_values(by=['lat', 'lng'])
nodes.index = range(nodes.shape[0])

In [4]:
df_police_stops = pd.read_csv('ca_san_francisco_2019_08_13.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
from scipy.spatial import distance as dist
from time import time

coords = nodes[['lng', 'lat']].values
batch_size = 1000
police_stops = {}
m = df_police_stops.shape[0]
for i in range(0, m, batch_size):
    if i % 10000 == 0: print('%d/%d' % (i, m))
    if i + batch_size >= m:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:]
    else:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:i+batch_size]
        
    associated_nodes = dist.cdist(coords, stop_coords).argmin(axis=0)
    for i in range(len(associated_nodes)):
        police_stops[associated_nodes[i]] = police_stops.get(associated_nodes[i], []) + [df_police_stops.iloc[i].values]

0/905070
10000/905070
20000/905070
30000/905070
40000/905070
50000/905070
60000/905070
70000/905070
80000/905070
90000/905070
100000/905070
110000/905070
120000/905070
130000/905070
140000/905070
150000/905070
160000/905070
170000/905070
180000/905070
190000/905070
200000/905070
210000/905070
220000/905070
230000/905070
240000/905070
250000/905070
260000/905070
270000/905070
280000/905070
290000/905070
300000/905070
310000/905070
320000/905070
330000/905070
340000/905070
350000/905070
360000/905070
370000/905070
380000/905070
390000/905070
400000/905070
410000/905070
420000/905070
430000/905070
440000/905070
450000/905070
460000/905070
470000/905070
480000/905070
490000/905070
500000/905070
510000/905070
520000/905070
530000/905070
540000/905070
550000/905070
560000/905070
570000/905070
580000/905070
590000/905070
600000/905070
610000/905070
620000/905070
630000/905070
640000/905070
650000/905070
660000/905070
670000/905070
680000/905070
690000/905070
700000/905070
710000/905070
720000

In [25]:
import pickle

with open('nodes_with_police_stops.pickle', 'wb') as handle:
    pickle.dump(police_stops, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('nodes_with_police_stops.pickle', 'rb') as handle:
    police_stops = pickle.load(handle)